# Installs

In [2]:
!pip install nltk
!pip install -U sentence_transformers
!pip install spacy==3.7.2
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.7.2/en_core_web_trf-3.7.2.tar.gz --no-deps
!pip install tabulate
!pip install pandas
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


# Imports

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
import json
import numpy as np
import pandas as pd
import spacy
import nltk
from nltk.tokenize import word_tokenize
from string import punctuation
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

[nltk_data] Error loading stopwords: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
c:\Users\sahaj\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the 

False

# Updating the Tool Set

To update the toolset, give information of new/updated tool in the following format given below and write in a json file. Pass the path of json file to the function ```update_toolset```.

In [ ]:
'''{
  <tool1_name>:{
     "Desription":" ",
     "ArgumentName":{
       <arg1_name>:{
         "Description":"",
         "ArgumentType": ""
        },
       <arg2_name>:{
         "Description":"",
         "ArgumentType": ""
        }
    }
  },
  <tool2_name>:{
     "Desription":" ",
     "ArgumentName":{
       <arg1_name>:{

         "Description":"",
         "ArgumentType": ""
        },
       <arg2_name>:{
         "Description":"",
         "ArgumentType": ""
        }
    }
  }
}'''

def update_toolset(new_tools,existing_toolset='tools_dictionary.json'):
  # new_tools : path to the json file containing new tools
  # existing_toolset: path of the existing tool_dictionary.json file
  with open(existing_toolset,"r") as f:
    parent_file=json.load(f)


  with open(new_tools,"r") as f1:
    new_file=json.load(f1)

  #adding the new tools
  parent_file.update(new_file)

  #making changes is the existing file only
  with open(existing_toolset,"w") as fo:
    json.dump(parent_file,fo,indent=4)




# Variable Initializations

In [5]:
address = "tools_dictionary.json"
with open(address, 'r') as file:
    data = json.load(file)

In [116]:
tools = []
tool_descriptions = []
arg_names = []
arg_descriptions = []
arg_to_tool_map = {}
tool_enc = []
tool_des_enc= []
arg_enc = []
arg_des_enc = []
no_of_block = 5
tool_embedding = []
arg_embedding = []

# Convert the dictionary to a list of descriptions without tool names
descriptions = [
    f"{data[tool]['Description']} - {data[tool]['ArgumentName'][arg]['Description']}"
    for tool in data
    for arg in data[tool]['ArgumentName']
]

# Print or use the list of concatenated descriptions
for desc in descriptions:
    print(desc)

# Different Sentence Transformer Models we are using for matching with different subqueries
models = [
    'all-MiniLM-L6-v2'
    ,'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
    ,'sentence-transformers/msmarco-MiniLM-L6-cos-v5'
    ,'sentence-transformers/paraphrase-MiniLM-L3-v2'
    ,'sentence-transformers/all-mpnet-base-v2'
    # ,'sentence-transformers/all-MiniLM-L6-v2'
    # ,'sentence-transformers/multi-qa-MiniLM-L6-cos-v1'
    # ,'sentence-transformers/paraphrase-albert-small-v2'
    ]

Given a text, extracts actionable insights, and creates tasks for them, which are kind of a work item. - The text from which the actionable insights need to be created.
Returns a list of work items that are similar to the given work item - The ID of the work item for which you want to find similar items
Returns the ID of the current sprint - _
Returns a list of objects sorted by priority. The logic of what constitutes priority for a given object is an internal implementation detail. - A list of objects to be prioritized
Given a search string, returns the id of a matching object in the system of record. If multiple matches are found, it returns the one where the confidence is highest. - The search string, could be for example customer’s name, part name, user name.
Summarizes a list of objects. The logic of how to summarize a particular object type is an internal
implementation
detail. - List of objects to summarize
Returns the ID of the current user - _
Returns a list of work items matc

# Utilities

In [35]:
# Given a list of strings and Sentence Transformer model it returns a list of vector embeddings
def embed_sentences(strings,model):
        embeddings = []
        for string in strings:
            embeddings.append(np.array(model.encode(string), dtype=np.float32))
        return np.array(embeddings)

'''
It extracts tool names, tool descriptions, argument names, argument descriptions
and puts them in a list for each type and also generates a dictionary mapping
each argument to its tool
'''
def extract():
    tools.clear()
    tool_descriptions.clear()
    arg_names.clear()
    arg_descriptions.clear()
    arg_to_tool_map.clear()

    for key, value in data.items():
        tmp_tool = key
        tools.append(tmp_tool)

        tmp_tool_dsc = value.get("Description")
        tool_descriptions.append(tmp_tool_dsc)

        tmp_arg_dsc = value.get("ArgumentName")

        if tmp_arg_dsc is not None:  # Check if tmp_arg_dsc is not None
            for argName, argDesc in tmp_arg_dsc.items():
                if argDesc.get("Description") == "_":
                    arg_names.append(key)
                    arg_descriptions.append(tmp_tool_dsc)
                    arg_to_tool_map[key] = key
                else:
                    arg_names.append(argName)
                    arg_descriptions.append(argDesc.get("Description"))
                    arg_to_tool_map[argName] = key



# Removes unnecessary stopwords like though, surely, etc. which could dilute the information in a sentence
def remove_stopwords(query):
        stop_words = set(stopwords.words('english'))
        word_tokens = word_tokenize(query)
        filtered_query = [word for word in word_tokens if word.lower() not in stop_words]
        return ' '.join(filtered_query)

# Generates embeddings
def generate_embeddings(model):
    tool_enc = embed_sentences(tools, model)
    tool_des_enc = embed_sentences(tool_descriptions,model)
    arg_enc = embed_sentences(arg_names,model)
    arg_des_enc = embed_sentences(arg_descriptions,model)

# Using spacy model to break the given query into multiple subqueries
def break_down_complex_sentence(sentence):
        # Load the spaCy model
        nlp = spacy.load('en_core_web_sm')
        doc = nlp(sentence)

        # Initialize variables
        clauses = []
        current_clause = []

        # Iterate through tokens in the sentence
        for token in doc:
            if token.dep_ in ['punct', 'cc'] and token.text != ',':
                # If it's a punctuation or coordinating conjunction (cc) that is not a comma,
                # consider it as the end of the current clause
                if current_clause:
                    clauses.append(' '.join(current_clause).strip())
                    current_clause = []
            elif token.text == ',':
                # If it's a comma, consider it as the end of the current clause,
                # and start a new clause if there are more tokens after the comma
                if current_clause:
                    clauses.append(' '.join(current_clause).strip())
                    current_clause = []
            else:
                # Otherwise, add the token to the current clause
                current_clause.append(token.text)

        # Add the last clause if there is any
        if current_clause:
            clauses.append(' '.join(current_clause).strip())

        return clauses

def tokenize_sentence(sentence):
    # Tokenize the sentence into words
    words = word_tokenize(sentence)

    # Remove punctuation marks
    words_without_punctuations = [word for word in words if word.isalnum()]

    return words_without_punctuations
    # Example usage
    input_sentence = "List all high severity tickets coming in from slack from customer Cust123 and generate a summary of them."
    result = tokenize_sentence(input_sentence)
    print(result)

In [38]:
def calculate_embeddings(models):
  extract()
  tmodels =[]
  tool_embedding.clear()
  arg_embedding.clear()
  for model in models:
    tmodels.append(SentenceTransformer(model))
  for model in tmodels:
    temp = embed_sentences(tool_descriptions,model)
    temp2 = embed_sentences(arg_descriptions, model)
    tool_embedding.append(temp)
    arg_embedding.append(temp2)


In [40]:
'''
This function takes the query, list of Sentence Transformer models, and minimum threshold
as arguments and implements our approach where it breaks down the query into multiple sub-queries
and uses different models to compute sentence embeddings of both the list of sub-queries and list of
tool descriptions. After calculating embeddings it generates a table of cosine similaritites
where each row is assigned with a particular tool and each column is assigned with a particular
subquery. From this table each column is analyzed to find which tool is giving maximum cosine similarity
for a specific subquery that information is stored and this process is repeated over multiple Sentence
Transformer models to calculate similar information. All this is consolidated into a final table where columns
again correspond to subqueries and rows correspond to model and rows will contain best tools faoured according to
the said model and from this table again a majority vote is taken to decide the final tools for each subquery.
In future we would avoid collisions for those tools which will contain a single argument,i.e., if two subqueries
are found be favouring same tool of said kind then the subquery demonastrating max cosine score with tool description
will be selected while the second subquery will have to go with next most favoured tool.
A threshold is provided to reject gibberish subqueries like "What is the meaning of my life?" which will produce a
low cosine similarity with any tool on an average.
'''
def tool_computer(query, models, threshold):
    sub_queries = break_down_complex_sentence(query)
    #print("Sub-Queries")
    #print(sub_queries)
    extract()
    #print("Subqueries:", sub_queries)
    tool_des = tool_descriptions
    #print(tool_des)
    '''for i in range(len(tool_des)):
      tool_des[i] = remove_stopwords(tool_des[i])
    for i in range(len(sub_queries)):
      sub_queries[i] = remove_stopwords(sub_queries[i])
    '''
    #print(tool_des)
    # Create a 2D list

    # Print the dimensions (number of rows and columns)
    final_table = []
    max_cs = None
    done = False
    for i, model in enumerate(models):
        temp_model = SentenceTransformer(model)
        table = np.zeros((len(tool_des),len(sub_queries)))
        for x in range(table.shape[0]):
            for y in range(table.shape[1]):
                table[x][y] = util.cos_sim(temp_model.encode(tool_des[x]),temp_model.encode(sub_queries[y]))
        if(done == False):
            done = True
            max_cs = np.max(table, axis=0)
        else:
            max_values = np.max(table, axis=0)
            for i in range(max_values.shape[0]):
                if(max_values[i] > max_cs[i]):
                    max_cs[i] = max_values[i]
        l = np.argmax(table, axis=0)
        #print("Model name: ",model)
        #print("Index of tools selected by it:")
        #print(l)
        final_table.append(l)
    #print("Final table:(From above information)")
    df = pd.DataFrame(final_table)
    #print(df)
    #stacked_array = np.vstack(final_table)

    # Find the unique values and their counts column-wise
    unique_values, counts = np.unique(final_table, axis=0, return_counts=True)

    # Find the index of the maximum count for each column
    max_count_indices = np.argmax(counts, axis=0)

    # Get the values with maximum frequency for each column
    max_freq_values = unique_values[max_count_indices]

    # Create a list of indices to remove
    indices_to_remove = [i for i, value in enumerate(max_cs) if value < threshold]

    # Remove elements from max_cs based on indices
    max_cs = [value for i, value in enumerate(max_cs) if i not in indices_to_remove]

    # Remove corresponding elements from max_freq_values
    max_freq_values = [value for i, value in enumerate(max_freq_values) if i not in indices_to_remove]


    #print("Number with maximum frequency column-wise:", max_freq_values)
    #print("Maximum Thresholds observed column-wise:",max_cs)
    max_freq_desc = list(zip(max_freq_values,max_cs))
    return max_freq_desc

In [111]:
def GetTools(query):
    x = tool_computer(input, models, 0.4)

    tool_list =[]
    for element in x:
        tool_list.append(tools[element[0]])

    # Initialize lists to store argument details
    argument_lists = []
    argument_descriptions = []
    argument_types = []

    # Iterate through each tool in the tool list
    for tool_name in tool_list:
        if tool_name in data:
            tool_data = data[tool_name]["ArgumentName"]
            arguments = list(tool_data.keys())
            descriptions = [tool_data[arg]["Description"] for arg in arguments]
            types = [tool_data[arg]["ArgumentType"] for arg in arguments]

            # Append to the respective lists
            argument_lists.append(arguments)
            argument_descriptions.append(descriptions)
            argument_types.append(types)
        else:
            # If the tool is not found in the data dictionary
            argument_lists.append([])
            argument_descriptions.append([])
            argument_types.append([])

    # Set up the question-answering model
    question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

    # Given data
    sub_queries = break_down_complex_sentence(input)

    argument_name = argument_lists
    argument_description = argument_descriptions
    argument_type = argument_types

    # Function to generate context for a given tool and sub-query
    def generate_context(tool, query, argument, argument_description, argument_type):
        context = f"{query} given tool used is {tool}."
        question = f"""{{
                        "{tool}": {{
                            "Description": "What is the argument value for the {argument} parameter in the {tool} tool?",
                            "ArgumentValue": {{
                                "{argument}": {{
                                    "Description": "{argument_description}",
                                    "ArgumentType": "{argument_type}"
                                }}
                            }}
                        }}
                    }}
                    """
        input_data = {
            "question": question,
            "context": context
        }
        result = question_answerer(input_data)
        return result['answer'], result['score']

    # Set threshold for filtering out answers
    threshold = 0.1

    # Lists to store filtered arguments with descriptions, types, and scores
    filtered_arguments = []
    filtered_argument_descriptions = []
    filtered_argument_types = []

    # Iterate through tools, sub_queries, and arguments
    for tool, query in zip(tool_list, sub_queries):
        # Find the index of the current tool
        tool_index = tool_list.index(tool)

        # Retrieve the corresponding argument information
        argument_list = argument_name[tool_index]
        arg_desc = argument_description[tool_index]
        arg_type = argument_type[tool_index]

        # Lists to store filtered sub-arguments for the current tool and query
        filtered_sub_arguments = []
        filtered_sub_arg_desc = []
        filtered_sub_arg_type = []

        # Iterate through sub-arguments if argument is a list
        for sub_argument, desc, arg_type in zip(argument_list, arg_desc, arg_type):
            answer, score = generate_context(tool, query, sub_argument, desc, arg_type)

            # Filter out based on the threshold
            if score >= threshold:
                filtered_sub_arguments.append(sub_argument)
                filtered_sub_arg_desc.append(desc)
                filtered_sub_arg_type.append(arg_type)
            else:
                # Append empty strings if no arguments were found
                filtered_sub_arguments.append(sub_argument)
                filtered_sub_arg_desc.append(' ')
                filtered_sub_arg_type.append(' ')

        # Append the filtered sub-arguments for the current tool and query
        if filtered_sub_arguments:
            filtered_arguments.append(filtered_sub_arguments)
            filtered_argument_descriptions.append(filtered_sub_arg_desc)
            filtered_argument_types.append(filtered_sub_arg_type)
        else:
            # Append empty strings if no arguments were found
            filtered_arguments.append(' ')
            filtered_argument_descriptions.append(' ')
            filtered_argument_types.append(' ')

    # Given query and tools
    arguments = filtered_arguments
    argument_description = filtered_argument_descriptions
    argument_types = filtered_argument_types

    # Initialize the workflow
    workflow = []

    # Set the threshold
    threshold = 0.3

    # Initialize the prev_index
    prev_index = -1

    # Process each sub-query and build the workflow
    for i, sub_query in enumerate(sub_queries):
        if i < len(tool_list):  # Ensure that the index is within the range of the tools list
            # Use the question-answering model to get the relevant information about the tool and its argument
            tool_name = tool_list[i]
            argument_names = arguments[i] if isinstance(arguments[i], list) else [arguments[i]]  # Handle the case where arguments is a string
            tool_arguments = []

            # Iterate through the sub-list of arguments
            for argument_name, arg_desc, arg_type in zip(argument_names, argument_description[i], argument_types[i]):
                # If the argument is empty, append an empty list to the workflow
                if argument_name.strip() == '':
                    tool_arguments = []  # Set tool_arguments to an empty list
                    break  # Break out of the loop as no further processing is needed for empty arguments
                else:
                    # Modify the question and context for the question-answering model
                    context = f"""
                    {sub_query} given tool used {tool_name}
                    """

                    question = f"""{{
                        "{tool_name}": {{
                            "Description": "What is the argument value for the {argument_name} parameter in the {tool_name} tool?",
                            "ArgumentValue": {{
                                "{argument_name}": {{
                                    "Description": "{arg_desc}",
                                    "ArgumentType": "{arg_type}"
                                }}
                            }}
                        }}
                    }}
                    """

                    # Provide the modified question and context as a dictionary
                    input_data = {
                        "question": question,
                        "context": context
                    }

                    # Get the answer using the question-answering model
                    result = question_answerer(input_data)

                    # Check the score and apply the threshold
                    if result['score'] < threshold:
                        # If the score is below the threshold, increment $$PREV[]
                        prev_index += 1
                        argument_value = "$$PREV[{}]".format(prev_index)
                        tool_arguments.append({"argument_name": argument_name, "argument_value": argument_value})
                    else:
                        # If the score is above the threshold, use the answer from the model
                        argument_value = result['answer']
                        tool_arguments.append({"argument_name": argument_name, "argument_value": argument_value})

            # Add the tool information with its arguments to the workflow
            workflow.append({"tool_name": tool_name, "arguments": tool_arguments})


    return workflow


# Run the test

In [118]:
queries = [
    'Get all work items similar to TKT-123, summarize them, create issues from that summary, and prioritize them',
    'Given a customer meeting transcript T, create action items and add them to my current sprint',
    'List all high severity tickets coming in from slack from customer Cust123 and generate a summary of them.',
    'What are my all issues in the triage stage under part FEAT-123? Summarize them.',
    'Summarize high severity tickets from the customer UltimateCustomer',
    'Prioritize my P0 issues and add them to the current sprint',
    'What is the meaning of life?',
    'Summarize issues similar to don:core:dvrv-us-1:devo/0:issue/1'
]
outputs = []
for input in queries:
    outputs.append(GetTools(input))

c:\Users\sahaj\miniconda3\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.7.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [124]:
outputs

[[{'tool_name': 'get_similar_work_items',
   'arguments': [{'argument_name': 'work_id', 'argument_value': 'TKT-123'}]},
  {'tool_name': 'summarize_objects',
   'arguments': [{'argument_name': 'objects', 'argument_value': '$$PREV[0]'}]},
  {'tool_name': 'summarize_objects',
   'arguments': [{'argument_name': 'objects', 'argument_value': '$$PREV[1]'}]},
  {'tool_name': 'prioritize_objects',
   'arguments': [{'argument_name': 'objects',
     'argument_value': '$$PREV[2]'}]}],
 [{'tool_name': 'create_actionable_tasks_from_text',
   'arguments': [{'argument_name': 'text', 'argument_value': '$$PREV[0]'}]},
  {'tool_name': 'add_work_items_to_sprint',
   'arguments': [{'argument_name': 'work_ids', 'argument_value': '$$PREV[1]'},
    {'argument_name': 'sprint_id', 'argument_value': '$$PREV[2]'}]}],
 [{'tool_name': 'summarize_objects',
   'arguments': [{'argument_name': 'objects',
     'argument_value': '$$PREV[0]'}]}],
 [{'tool_name': 'summarize_objects',
   'arguments': [{'argument_name': 'obj

# Evaluation

To do the evaluation, we are using BLEU Metric. To do meaning full evaluation, the format of the hypothesis and reference strings should be similar else the results are uninterpretable.

In [125]:
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu

##### Example 1: Query - Given a customer meeting transcript T, create action items and add them to my current sprint

In [126]:
hypothesis = """
[
    {
    "tool_name": "create_actionable_tasks_from_text",
    "arguments": [
            {
    "argument_name": "text",
    "argument_value": "T"
            }
        ]
    },
    {
    "tool_name": "get_sprint_id",
    "arguments": []
    },
    {
    "tool_name": "add_work_items_to_sprint",
    "arguments": [
            {
    "argument_name": "work_ids",
    "argument_value": "$$PREV[0]"
            },
            {
    "argument_name": "sprint_id",
    "argument_value": "$$PREV[1]"
            }
        ]
    }
]
"""
reference = """
[
  {
    "tool_name": "create_actionable_tasks_from_text",
    "arguments": [
      {
        "argument_name": "text",
        "argument_value": "$$PREV[0]"
      }
    ]
  },
  {
    "tool_name": "add_work_items_to_sprint",
    "arguments": [
      {
        "argument_name": "work_ids",
        "argument_value": "$$PREV[1]"
      },
      {
        "argument_name": "sprint_id",
        "argument_value": "$$PREV[2]"
      }
    ]
  }
]
"""

In [127]:
BLEU_5 = sentence_bleu([reference], hypothesis, weights=(0.2, 0.2, 0.2, 0.2, 0.2))
print(f"\nThe score for BLEU-5 is: {BLEU_5}")


The score for BLEU-5 is: 0.8101603728876992


##### Example 2: Query - Get all work items similar to TKT-123, summarize them, create issues from that summary, and prioritize them

In [128]:
hypothesis = """
[
  {
    "tool_name": "get_similar_work_items",
    "arguments": [
      {
        "argument_name": "work_id",
        "argument_value": "TKT-123"
      }
    ]
  },
  {
    "tool_name": "summarize_objects",
    "arguments": [
      {
        "argument_name": "objects",
        "argument_value": "$$PREV[0]"
      }
    ]
  },
  {
    "tool_name": "summarize_objects",
    "arguments": [
      {
        "argument_name": "objects",
        "argument_value": "$$PREV[1]"
      }
    ]
  },
  {
    "tool_name": "prioritize_objects",
    "arguments": [
      {
        "argument_name": "objects",
        "argument_value": "$$PREV[2]"
      }
    ]
  }
]
"""
reference = """
[
  {
    "tool_name": "get_similar_work_items",
    "arguments": [
      {
        "argument_name": "work_id",
        "argument_value": "TKT-123"
      }
    ]
  },
  {
    "tool_name": "summarize_objects",
    "arguments": [
      {
        "argument_name": "objects",
        "argument_value": "$$PREV[0]"
      }
    ]
  },
  {
    "tool_name": "create_actionable_tasks_from_text",
    "arguments": [
      {
        "argument_name": "text",
        "argument_value": "$$PREV[1]"
      }
    ]
  },
  {
    "tool_name": "prioritize_objects",
    "arguments": [
      {
        "argument_name": "objects",
        "argument_value": "$$PREV[2]"
      }
    ]
  }
]
"""

In [129]:
BLEU_5 = sentence_bleu([reference], hypothesis, weights=(0.2, 0.2, 0.2, 0.2, 0.2))
print(f"\nThe score for BLEU-5 is: {BLEU_5}")


The score for BLEU-5 is: 0.9439792470039666


##### Example 3: Query - List all high severity tickets coming in from slack from customer Cust123 and generate a summary of them

In [131]:
hypothesis = """
[
  {
    "tool_name": "summarize_objects",
    "arguments": [
      {
        "argument_name": "objects",
        "argument_value": "$$PREV[0]"
      }
    ]
  }
]
"""
reference = """
[
  {
    "tool_name": "search_object_by_name",
    "arguments": [
      {
        "argument_name": "query",
        "argument_value": "Cust123"
      }
    ]
  },
  {
    "tool_name": "works_list",
    "arguments": [
      {
        "argument_name": "ticket.rev_org",
        "argument_value": [
          "$$PREV[0]"
        ]
      },
      {
        "argument_name": "ticket.severity",
        "argument_value": [
          "high"
        ]
      },
      {
        "argument_name": "ticket.source_channel",
        "argument_value": [
          "slack"
        ]
      },
      {
        "argument_name": "type",
        "argument_value": [
          "ticket"
        ]
      }
    ]
  },
  {
    "tool_name": "summarize_objects",
    "arguments": [
      {
        "argument_name": "objects",
        "argument_value": "$$PREV[1]"
      }
    ]
  }
]
"""

In [132]:
BLEU_5 = sentence_bleu([reference], hypothesis, weights=(0.2, 0.2, 0.2, 0.2, 0.2))
print(f"\nThe score for BLEU-5 is: {BLEU_5}")


The score for BLEU-5 is: 0.015472752599220563
